# PSQL to Python `User Defined Functions`: `TRIGGERS`

**<font color=red> Mr Fugu Data Science</font>**

# (◕‿◕✿)

[Github](https://github.com/MrFuguDataScience) | [Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw/)

# Purpose & Outcome:

+ Create a new table, to store our trigger information
    + Create a Function to interface between table of interest and store changed data that will go into new table.

+ `CREATE A TRIGGER` and explain what it is and how to use




# Install Psycopg2:

`python -m pip install psycopg2`

if this doesn't work, try changing `pip to pip3` depending on what version of Python you are using

if that doesn't work either try to do: `conda install -c anaconda psycopg2`

In [108]:
import psycopg2             # python->psql connection
import psycopg2.extras

import pandas as pd   


# Import the 'config' function from the config_user_dta.py file:
from config_user_dta import config

# `Triggers`: *This came up on a Job Interview Test*  

# 😟 `this was my face`
( *I had 1 minute to figure it out* )

+ What is a `Trigger`? : think of a function the user creates that will be associated or 'bound' to a specific table in your current database.
    + It will automatically be used when a specific event occurs, that you define.
        + An event: consists of (`Insert, Update, Delete, Truncate`) for example. And can be used either `Before, After, or Instead of` one of these events occuring. 

`-------------------------------`

+ Two Types `(Row level,Statement level)`
 + The difference, `Row` wise is done on a per row basis and when it is executed. 

`__________________________________________________`

+ **When can** `User Defined Functions AKA Stored Procedures` **be used**?
 + This is a case by case situation.
     If you have a scenario where you are frequently calling thousands of rows frequently this can save you time. 
     
+ Consider if you need to make several trips to the database making multiple queries. This can be handled in a function defined by the user all at once. 

+ `Data Warehousing`: uses these techniques to reference data from a tables such as a summary is an example. 


**General Layout**:

`CREATE TRIGGER name_of_trigger
{BEFORE | AFTER | INSTEAD OF} {event [OR ...]}
   ON table_name
   [FOR [EACH] {ROW | STATEMENT}]
       EXECUTE PROCEDURE trigger_function
`


[Details of Triggers](https://www.tutorialspoint.com/postgresql/postgresql_triggers.htm)

In [2]:
# Establish a connection to the database by creating a cursor object

# Get the config params
params_ = config()

# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)

# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()

conn.autocommit = True  # read documentation understanding when to Use & NOT use (TRUE)


# Create a table used for our `Trigger`:

In [99]:
# CREATE TABLE FOR PSQL: cpu_sales_update (used to log updated sales)

def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS cpu_sales_update;
        CREATE UNLOGGED TABLE cpu_sales_update (
            changed_on TIMESTAMP(6) NOT NULL,
            credit_card         TEXT NOT NULL,
            email               TEXT NOT NULL,
            first_name          TEXT NOT NULL,
            last_name           TEXT NOT NULL,
            primary_phone       TEXT NOT NULL,
            cpu                 TEXT NOT NULL,
            purchase_date       DATE NOT NULL
        );""")
    


In [100]:
with conn.cursor() as cursor:
    create_staging_table(cursor)


# Create Function to Log Updated Sales

+ such as changed cpu,credit info


In [72]:
def create_log_func(cursor):
    cursor.execute("""
    CREATE OR REPLACE FUNCTION log_sales_data()
    RETURNS TRIGGER AS $cpu_sales_update$
        BEGIN
            --
            -- Create a new row in cpu_sales_update, for operation performed on 
            -- 'ppl_cpu_purchases'. (TG_OP) is used to designate what operation is
            -- performed and triggers your 'TRIGGER'

            IF (TG_OP = 'DELETE') THEN 
                INSERT INTO cpu_sales_update SELECT now(), OLD.*;
                RETURN OLD;

            ELSEIF (TG_OP = 'UPDATE') THEN
                INSERT INTO cpu_sales_update SELECT now(),NEW.*;
                RETURN NEW;

            ELSIF (TG_OP = 'INSERT') THEN
                INSERT INTO cpu_sales_update SELECT now(),NEW.*;

            END IF;
            RETURN NULL; -- result is ignored since this is an AFTER trigger
        
        END;    
    $cpu_sales_update$ LANGUAGE plpgsql;""")

In [73]:
# Send Over for Function which will store trigger information
with conn.cursor() as cursor:
    create_log_func(cursor)


In [74]:
# Trigger Creation: which will start (AFTER) the event!

def trigger_cpu_sales(cursor):
    cursor.execute("""
    CREATE TRIGGER cpu_sales_update_trigger
    AFTER INSERT OR UPDATE OR DELETE ON ppl_cpu_purchases
    FOR EACH ROW EXECUTE PROCEDURE log_sales_data();
    """)


In [75]:
# Send Trigger to PSQL:

with conn.cursor() as cursor:
    trigger_cpu_sales(cursor)

DuplicateObject: trigger "cpu_sales_update_trigger" for relation "ppl_cpu_purchases" already exists


In [64]:
# All Triggers in Current DB:

cur.execute('SELECT * FROM pg_trigger;')
cur.fetchall()

[(18454,
  16586,
  'cpu_sales_update_trigger',
  18418,
  29,
  'O',
  False,
  0,
  0,
  0,
  False,
  False,
  0,
  '',
  None,
  None,
  None)]

In [65]:
# Find What Triggers are on a specific Table:

cur.execute("SELECT tgname FROM pg_trigger, \
pg_class WHERE tgrelid=pg_class.oid AND relname='ppl_cpu_purchases';")

cur.fetchall()

[('cpu_sales_update_trigger',)]

In [105]:
cur.execute("SELECT tgname FROM pg_trigger, \
pg_class WHERE tgrelid=pg_class.oid AND relname='fake_r_users';")

cur.fetchall()

[]

In [107]:
# Drop Trigger, I first dropped the table with it that I created.


# cur.execute("DROP TRIGGER IF EXISTS cpu_sales_update ON ppl_cpu_purchases;")
# cur.execute("DROP TABLE IF EXISTS cpu_sales_update;")

In [76]:
# Do an Insert and make sure Trigger works:

cur.execute("INSERT INTO ppl_cpu_purchases (credit_card,email,first_name,last_name, \
primary_phone,cpu,purchase_date) VALUES ('9999-9999-9999-9999','MrFugu@gmail.com', \
'MrFugu','DataScience','1111111111', 'Intel Core i9-9999K','2020-05-05')")


In [91]:
cur.execute('SELECT * FROM cpu_sales_update')
cur.fetchall()

[(datetime.datetime(2020, 6, 16, 17, 51, 46, 698363),
  '9999-9999-9999-9999',
  'MrFugu@gmail.com',
  'MrFugu',
  'DataScience',
  '1111111111',
  'Intel Core i9-9999K',
  datetime.date(2020, 5, 5))]

In [101]:
cur.execute("UPDATE ppl_cpu_purchases SET first_name= 'MrFuguYay' \
WHERE first_name='MrFugu'")

In [102]:
cur.execute('SELECT * FROM cpu_sales_update')
cur.fetchall()

[(datetime.datetime(2020, 6, 16, 18, 27, 20, 610811),
  '9999-9999-9999-9999',
  'MrFugu@gmail.com',
  'MrFuguYay',
  'DataScience',
  '1111111111',
  'Intel Core i9-9999K',
  datetime.date(2020, 5, 5))]

# Citations & Useful Help:

# ◔̯◔

https://www.postgresqltutorial.com/creating-first-trigger-postgresql/

https://www.postgresqltutorial.com/postgresql-stored-procedures/

https://www.tutorialspoint.com/postgresql/postgresql_functions.htm

https://www.tutorialspoint.com/postgresql/postgresql_triggers.htm

https://www.postgresql.org/docs/9.2/plpgsql-trigger.html

https://www.postgresqltutorial.com/plpgsql-function-overloading/

https://www.postgresql.org/docs/9.5/trigger-example.html (Written in C)

https://medium.com/simform-engineering/what-when-and-how-of-database-triggers-using-postgresql-f4464bd969d4

https://pynative.com/python-execute-postgresql-stored-procedure-and-function/

https://www.codementor.io/@engineerapart/getting-started-with-postgresql-on-mac-osx-are8jcopb

https://w3resource.com/PostgreSQL/postgresql-triggers.php
